In [2]:
import torch
import numpy as np
import torch.nn.functional as F

def select_confident_samples(logits, top):
    batch_entropy = -(logits.softmax(1) * logits.log_softmax(1)).sum(1)
    idx = torch.argsort(batch_entropy, descending=False)[:int(batch_entropy.size()[0] * top)]
    return logits[idx], idx

def avg_entropy(outputs):
    # epsilon = 1e-10
    assert len(outputs) > 0
    assert torch.any(torch.isnan(outputs)) == False
    # logits = outputs - outputs.logsumexp(dim=-1, keepdim=True) # logits = outputs.log_softmax(dim=1) [N, 1000]
    logits = outputs.log_softmax(dim=-1) #[N, 1000]
    assert torch.any(torch.isnan(logits)) == False
    # avg_logits = logits.logsumexp(dim=0) - np.log(logits.shape[0]) # avg_logits = logits.mean(0) [1, 1000]
    avg_logits = logits.mean(0) #[1, 1000]
    # print(avg_logits)
    if torch.any(torch.isnan(avg_logits)):
        print("average logits ", outputs.log_softmax(dim=1).mean(0))
    assert torch.any(torch.isnan(avg_logits)) == False
    
    min_real = torch.finfo(avg_logits.dtype).min
    avg_logits = torch.clamp(avg_logits, min=min_real)
    assert torch.any(torch.isnan(avg_logits)) == False
    return -((avg_logits) * (torch.exp(avg_logits))).sum(dim=-1)

kl_loss = torch.nn.KLDivLoss(reduction="batchmean")




/home/seongha/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

logit = torch.tensor([12.0, 3, 3, 3.0], requires_grad=True)
logit2 = torch.tensor([3, 3, 3, 12.0], requires_grad=True)
GT = torch.tensor([3, 3, 3, 9.0], requires_grad=True)
for each in [logit, logit2]:
    print("logit ", each)
    print("target dist ", GT)
    print(avg_entropy(each))
    # print(avg_entropy(torch.nn.functional.softmax(each, dim=-1)))
    
    input = F.log_softmax(each, dim=-1)
    # input = F.log_softmax(torch.nn.functional.softmax(each, dim=-1), dim=-1)
    target = F.softmax(GT, dim=-1)
    one = 0.5 *kl_loss(input, target)
    print(input)
    print("kl loss: ", kl_loss(input, target))

    input = F.log_softmax(GT, dim=-1)
    # input = F.log_softmax(torch.nn.functional.softmax(GT, dim=-1), dim=-1)
    target = F.softmax(each, dim=-1)
    two = 0.5 *kl_loss(input, target)
    print("kl loss: ", kl_loss(input, target))
    print("JS loss : {}\n".format( one + two))
    
    

logit  tensor([12.,  3.,  3.,  3.])
target dist  tensor([3., 3., 3., 9.])
tensor(0.0079)
tensor([-3.7008e-04, -9.0004e+00, -9.0004e+00, -9.0004e+00])
kl loss:  tensor(2.2316)
kl loss:  tensor(1.5007)
JS loss : 1.8661872148513794

logit  tensor([ 3.,  3.,  3., 12.])
target dist  tensor([3., 3., 3., 9.])
tensor(0.0079)
tensor([-9.0004e+00, -9.0004e+00, -9.0004e+00, -3.7020e-04])
kl loss:  tensor(0.0038)
kl loss:  tensor(0.0015)
JS loss : 0.0026292267721146345

